In [1]:
import sys
import copy
import random
import os
from typing import Tuple
from collections import OrderedDict

import numpy as np
import PIL 
import matplotlib.pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# import tensorflow as tf

import torch
from torch import nn, optim
from torch.utils import data
from torchvision import datasets, transforms

In [2]:
# sys.path.append("/home/matthias/Documents/EmbeddedAI/deep-microcompression/")
sys.path.append("../../")

from development import (
    Sequential,
    AvgPool2d,
    BatchNorm2d,
    Conv2d,
    Flatten,
    Linear,
    ReLU6,

    EarlyStopper,

    QuantizationGranularity,
    QuantizationScheme
)

/home/matthias/Documents/EmbeddedAI/deep-microcompression/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
mobilenetv1_file = f"mobilenetv1_state_dict_from_tf_{DEVICE}.pth"
mobilenetv1_state_dict_dmc_original_from_tf = f"mobilenetv1_state_dict_dmc_original_from_tf.pth"

input_shape = (3, 224, 224)
LUCKY_NUMBER = 25
torch.manual_seed(LUCKY_NUMBER)
torch.random.manual_seed(LUCKY_NUMBER)
torch.cuda.manual_seed(LUCKY_NUMBER)

DEVICE = "cpu"

## Getting the Datasets

In [4]:

class ImageNet_Validation_DataSet(data.Dataset):

    def __init__(self, image_dir, combined_label_file, transformer=None):
        
        assert os.path.exists(image_dir), f"image_dir {image_dir} doesn't exist"
        assert os.path.exists(combined_label_file), f"combined_label_file {combined_label_file} doesn't exist!"

        self.image_dir = image_dir
        self.images =  os.listdir(image_dir)
        self.images.sort()

        with open(combined_label_file, "r") as file:
            self.labels, self.class_names = list(), list()
            for line in file.readlines()[1:]:
                _, _, tf_label, class_name = line.strip().split(", ")
                self.labels.append(tf_label)
                self.class_names.append(class_name)

        assert len(self.images) == len(self.labels), "Images not of the same length as targets"
        self.transformer = transformer
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = PIL.Image.open(os.path.join(self.image_dir, self.images[idx])).convert("RGB")
        if self.transformer:
            image = self.transformer(image)
        return  image, int(self.labels[idx])
        
imagenet_transformer = transforms.Compose([
    transforms.Resize(input_shape[1:]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.50], std=[0.5, 0.5, 0.5])
])

imagenet_val_dir = "../../../Datasets/ImageNet_2012/validation/ILSVRC2012_img_val/"
imagenet_val_combined_label_file = "../../../Datasets/ImageNet_2012/validation/ILSVRC2012_validation_combined_ground_truth.txt"
imagenet_val_dataset = ImageNet_Validation_DataSet(image_dir=imagenet_val_dir, combined_label_file=imagenet_val_combined_label_file, transformer=imagenet_transformer)
imagenet_val_dataloader = data.DataLoader(imagenet_val_dataset, shuffle=False, batch_size=32, num_workers=os.cpu_count())

In [5]:
data_transform = transforms.Compose([
    # transforms.RandomCrop((24, 24)),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

cifar10_train_dataset = datasets.CIFAR10("../../../Datasets/CIFAR_10/", train=True, download=True, transform=data_transform)
cifar10_test_dataset = datasets.CIFAR10("../../../Datasets/CIFAR_10/", train=False, download=True, transform=data_transform)

cifar10_train_loader = data.DataLoader(cifar10_train_dataset, batch_size=32, shuffle=True)
cifar10_test_loader = data.DataLoader(cifar10_test_dataset, batch_size=32)

# cifar100_train_dataset = datasets.CIFAR100("./datasets", train=True, download=True, transform=data_transform)
# cifar100_test_dataset = datasets.CIFAR100("./datasets", train=False, download=True, transform=data_transform)

# cifar100_train_loader = data.DataLoader(cifar100_train_dataset, batch_size=32, shuffle=True, num_workers=os.cpu_count())
# cifar100_test_loader = data.DataLoader(cifar100_test_dataset, batch_size=32, num_workers=os.cpu_count())

## Building the model and Loading the Saved Weights
## Defining the Mobilenetv1 Structure

In [6]:
def MobileNetV1(load_tf_weights=True):

    def ConvBatchReLU(
            in_channels:int,
            out_channels:int,
            kernel_size:int,
            stride:int = 1,
            groups:int = 1,
            pad:Tuple[int, int, int, int] = (0, 0, 0, 0),
            bias=False,
            eps=0.001, 
            momentum=0.01,
    ):
        return (Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, groups=groups, pad=pad, bias=bias),
                BatchNorm2d(num_features=out_channels, eps=eps, momentum=momentum, affine=True, track_running_stats=True,),
                ReLU6(inplace=True))
    
    def DepthwiseSeperableConv2LUBatchReLU(
        in_channels:int,
        out_channels:int,
        kernel_size:int,
        stride:int,
        pad:Tuple[int, int, int, int] = (0, 0, 0, 0),
        eps=0.001, 
        momentum=0.01
    ):
        return (
            *ConvBatchReLU(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, stride=stride, pad=pad, groups=in_channels, eps=eps, momentum=momentum),
            *ConvBatchReLU(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, pad=(0,0,0,0), groups=1, eps=eps, momentum=momentum)
        )
    mobilenetv1 = {
        "conv2d_0": [3, 32, 3, 2],

        "depthwiseseparable_0": [32, 64, 3, 1],
        "depthwiseseparable_1": [64, 128, 3, 2],
        "depthwiseseparable_2": [128, 128, 3, 1],
        "depthwiseseparable_3": [128, 256, 3, 2],
        "depthwiseseparable_4": [256, 256, 3, 1],
        "depthwiseseparable_5": [256, 512, 3, 2],

        "depthwiseseparable_6": [512, 512, 3, 1],
        "depthwiseseparable_7": [512, 512, 3, 1],
        "depthwiseseparable_8": [512, 512, 3, 1],
        "depthwiseseparable_9": [512, 512, 3, 1],
        "depthwiseseparable_10": [512, 512, 3, 1],

        "depthwiseseparable_11": [512, 1024, 3, 2],
        "depthwiseseparable_12": [1024, 1024, 3, 1],

        "avgpool_0": [7],
        "conv2d_1": [1024, 1000, 1, 1],
        "flatten_0": []
    }

    batchnorm_eps = 0.001
    batchnorm_momentum = 1 - 0.99

    layers = []

    for name, parameters in mobilenetv1.items():
        if "conv2d_0" in name:
            in_channels, out_channels, kernel_size, stride = parameters
            if stride == 2:
                pad = (0, 1, 0, 1)
            else:
                raise RuntimeError(f"Unexpected type Conv layer")
            layers.extend(ConvBatchReLU(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, pad=pad, eps=batchnorm_eps, momentum=batchnorm_momentum))
        elif "conv2d_1" in name:
            in_channels, out_channels, kernel_size, stride = parameters
            layers.append(Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride))
        elif "depthwiseseparable" in name:
            in_channels, out_channels, kernel_size, stride = parameters
            if stride == 2:
                pad = (0, 1, 0, 1)
            else:
                pad = tuple([1]*4)
            layers.extend(DepthwiseSeperableConv2LUBatchReLU(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, pad=pad, eps=batchnorm_eps, momentum=batchnorm_momentum))
        elif "avgpool" in name:
            kernel_size = parameters[0]
            layers.append(AvgPool2d(kernel_size=kernel_size))
        elif "flatten" in name:
            layers.append(Flatten())
        else:
            raise ValueError(f"Recieved unexpected layer of {name}")
    mobilenetv1_model = Sequential(*layers)
    if load_tf_weights:
        mobilenetv1_model.load_state_dict(torch.load(mobilenetv1_state_dict_dmc_original_from_tf, weights_only=True), strict=True)
    return mobilenetv1_model

In [7]:
top1_acc_fun = lambda y_pred, y_true: ((y_pred).argmax(dim=1) == y_true).sum().item()    
top5_acc_fun = lambda y_pred, y_true: (y_pred.topk(5, dim=1).indices == y_true.unsqueeze(1)).any(dim=1).sum().item()

In [ ]:
mobilenetv1_model = MobileNetV1()
mobilenetv1_model.to(DEVICE)

original_acc1 = mobilenetv1_model.evaluate(imagenet_val_dataloader, top1_acc_fun, DEVICE)*100
# original_acc5 = mobilenetv1_model.evaluate(imagenet_val_dataloader, top5_acc_fun, DEVICE)*100
original_size = mobilenetv1_model.get_size_in_bytes()
original_size, original_acc1, original_acc5

Sequential(
  (conv2d_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (batchnorm2d_0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (relu6_0): ReLU6(inplace=True)
  (conv2d_1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
  (batchnorm2d_1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (relu6_1): ReLU6(inplace=True)
  (conv2d_2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (batchnorm2d_2): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (relu6_2): ReLU6(inplace=True)
  (conv2d_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), groups=64, bias=False)
  (batchnorm2d_3): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (relu6_3): ReLU6(inplace=True)
  (conv2d_4): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bi

## Testing Compression on MobileNetV1

### Pruning

In [ ]:
for i in range(0, 11, 1):
    sp = i/10
    compression_config = {
        "prune_channel" :{
            "sparsity" : sp,
            "metric" : "l2"
        },
    }
    compressed_mobilenetv1_mcu_model = mobilenetv1_model.init_compress(compression_config, input_shape=input_shape)

    before_acc1 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top1_acc_fun, device=DEVICE)*100
    before_acc5 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top5_acc_fun, device=DEVICE)*100
    size = compressed_mobilenetv1_mcu_model.get_size_in_bytes()
    print(f"Before training, sparsity = {sp} acc1 = {before_acc1:.4f} acc5 = {before_acc5:.4f} size = {size/original_size*100:9.4f}")


### Dynamic Quantization Per Tensor

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.DYNAMIC,
            "granularity": QuantizationGranularity.PER_TENSOR,
            "bitwidth" : b
        }
    }
    compressed_mobilenetv1_mcu_model = mobilenetv1_model.init_compress(compression_config, input_shape=input_shape)

    before_acc1 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top1_acc_fun, device=DEVICE)*100
    before_acc5 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top5_acc_fun, device=DEVICE)*100
    size = compressed_mobilenetv1_mcu_model.get_size_in_bytes()
    print(f"Before training, bitwidth = {b} acc1 = {before_acc1:.4f} acc5 = {before_acc5:.4f} size = {size/original_size*100:9.4f}")


100%|██████████| 1563/1563 [44:22<00:00,  1.70s/it]


NameError: name 'sp' is not defined

### Dynamic Quantization Per Channel

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.DYNAMIC,
            "granularity": QuantizationGranularity.PER_CHANNEL,
            "bitwidth" : b
        }
    }
    compressed_mobilenetv1_mcu_model = mobilenetv1_model.init_compress(compression_config, input_shape=input_shape)

    before_acc1 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top1_acc_fun, device=DEVICE)*100
    before_acc5 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top5_acc_fun, device=DEVICE)*100
    size = compressed_mobilenetv1_mcu_model.get_size_in_bytes()
    print(f"Before training, bitwidth = {b} acc1 = {before_acc1:.4f} acc5 = {before_acc5:.4f} size = {size/original_size*100:9.4f}")


### Static Quantization Per Tensor

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.STATIC,
            "granularity": QuantizationGranularity.PER_TENSOR,
            "bitwidth" : b
        }
    }
    compressed_mobilenetv1_mcu_model = mobilenetv1_model.init_compress(compression_config, input_shape=input_shape, calibration_data=next(iter(imagenet_val_dataloader))[0].to(DEVICE))

    before_acc1 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top1_acc_fun, device=DEVICE)*100
    before_acc5 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top5_acc_fun, device=DEVICE)*100
    size = compressed_mobilenetv1_mcu_model.get_size_in_bytes()
    print(f"Before training, bitwidth = {b} acc1 = {before_acc1:.4f} acc5 = {before_acc5:.4f} size = {size/original_size*100:9.4f}")


### Static Quantization Per Channel

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.STATIC,
            "granularity": QuantizationGranularity.PER_CHANNEL,
            "bitwidth" : b
        }
    }
    compressed_mobilenetv1_mcu_model = mobilenetv1_model.init_compress(compression_config, input_shape=input_shape, calibration_data=next(iter(imagenet_val_dataloader))[0].to(DEVICE))

    before_acc1 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top1_acc_fun, device=DEVICE)*100
    before_acc5 = compressed_mobilenetv1_mcu_model.evaluate(imagenet_val_dataloader, top5_acc_fun, device=DEVICE)*100
    size = compressed_mobilenetv1_mcu_model.get_size_in_bytes()
    print(f"Before training, bitwidth = {b} acc1 = {before_acc1:.4f} acc5 = {before_acc5:.4f} size = {size/original_size*100:9.4f}")


# MobileNetV1 Tensorflow to Torch Converter 

In [ ]:
@torch.no_grad()
def convert_mobilenetv1_tf_to_torch(save_weight=True):

    def copy_tensor(tensor_source, tensor_destination):
        tensor_destination.copy_(tensor_source)

    mobilenetv1_tf_model = tf.keras.applications.MobileNet(weights="imagenet")
    torch_layers = []

    for i, layer in enumerate(mobilenetv1_tf_model.layers):
    
        if isinstance(layer, tf.keras.layers.InputLayer):
            pass

        elif isinstance(layer, tf.keras.layers.Conv2D):
            # print(layer.weights[0].shape)

            weight = np.transpose(layer.weights[0], (3, 2, 0, 1))
            out_channels, in_channels, kernel_size, _ = weight.shape
            stride =layer.strides[0]

            if kernel_size == 3 and stride == 2:
                pad = (0, 1, 0, 1)
                padding = 0
            elif kernel_size == 1 and stride == 1:
                pad = (0,0,0,0)
                padding = 0
            else:
                raise RuntimeError(f"Unexpected type Conv layer")

            # torch_layers.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=layer.strides[0], padding=1 bias=True))
            torch_layers.append(Conv2d(in_channels=in_channels, out_channels=out_channels, 
            kernel_size=kernel_size, stride=stride, padding=padding, pad=pad, 
            bias=len(layer.weights)==2))
            copy_tensor(torch.from_numpy(weight), torch_layers[-1].weight)
                
            if len(layer.weights) == 2:
                # nn.init.constant_(torch_layers[-1].bias, 0.0)
            # else:
                copy_tensor(torch.from_numpy(layer.weights[1].numpy()), torch_layers[-1].bias)

            pass

        elif isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            # Convert to PyTorch format: [in_channels, 1, H, W]
            weight = np.transpose(layer.weights[0], (2, 3, 0, 1))
            
            in_channels, depth_multiplier, kernel_size, _ = weight.shape
            stride =layer.strides[0]

            assert depth_multiplier == 1, "Depth multiplier must be 1 for depthwise convolutions"

            if kernel_size == 3 and stride == 2:
                padding = 0
                pad = (0, 1, 0, 1)
            elif kernel_size == 3 and stride == 1:
                padding = 0
                pad = (1,1,1,1)
            else:
                raise RuntimeError(f"Unexpected type Conv layer")

            # torch_layers.append(nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, stride=layer.strides[0], padding=1, groups=in_channels, bias=True))
            torch_layers.append(
        Conv2d(in_channels=in_channels, out_channels=in_channels, 
        kernel_size=kernel_size, stride=stride, padding=padding, pad=pad, 
        groups=in_channels, bias=len(layer.weights)==2))
            print(i, weight.shape, torch_layers[-1].weight.shape, in_channels)

            # Copy weights
            copy_tensor(torch.from_numpy(weight), torch_layers[-1].weight)
            # nn.init.constant_(torch_layers[-1].bias, 0.0)
            pass

        elif isinstance(layer, tf.keras.layers.ReLU):
            # torch_layers.append(nn.ReLU())
            torch_layers.append(ReLU6())
            pass

        elif isinstance(layer, tf.keras.layers.BatchNormalization):
            gamma, beta, mean, var = layer.weights
            out_channels = gamma.shape[0]

            # torch_layers.append(nn.BatchNorm2d(out_channels, eps=layer.epsilon, momentum=layer.momentum))
            torch_layers.append(BatchNorm2d(out_channels, eps=layer.epsilon, momentum=1 - layer.momentum, affine=layer.center and layer.scale,  track_running_stats=True,))
            # print(torch_layers[-1].momentum)  
            # print(layer.epsilon, layer.momentum, layer.center, layer.scale)
            copy_tensor(torch.from_numpy(gamma.numpy()), torch_layers[-1].weight)
            copy_tensor(torch.from_numpy(beta.numpy()), torch_layers[-1].bias)
            copy_tensor(torch.from_numpy(mean.numpy()), torch_layers[-1].running_mean)
            copy_tensor(torch.from_numpy(var.numpy()), torch_layers[-1].running_var)

            # print(torch_layers[-1].running_var, layer.weights)

            # print(var)
            # print(layer.moving_variance)

            # break
            pass
            
        elif isinstance(layer, tf.keras.layers.Dense):
            weight, bias = layer.weights
            # torch_layers.append(nn.Linear(weight.shape[0], weight.shape[1]))
            torch_layers.append(Linear(weight.shape[0], weight.shape[1]))

            copy_tensor(torch.from_numpy(weight.numpy().T), torch_layers[-1].weight)
            copy_tensor(torch.from_numpy(bias.numpy()), torch_layers[-1].bias)

            # print(weight, bias)
            pass

        elif isinstance(layer, tf.keras.layers.GlobalAveragePooling2D):
            torch_layers.append(AvgPool2d(kernel_size=7))
            # torch_layers.append(MaxPool2d(kernel_size=7))
            pass

        elif isinstance(layer, tf.keras.layers.ZeroPadding2D):
            pass
        elif isinstance(layer, tf.keras.layers.Dropout):
            pass

        elif isinstance(layer, tf.keras.layers.Reshape):
            # print("Not Needed")
            # torch_layers.append(Flatten())
            pass

        elif isinstance(layer, tf.keras.layers.Activation):
            # torch_layers.append(nn.Softmax(dim=1))
            pass

        else: 
            pass
            raise RuntimeError(f"Unknown layer type: {type(layer)}")
    mobilenetv1_model = Sequential(*torch_layers)
    if save_weight:
        torch.save(mobilenetv1_model.state_dict(), mobilenetv1_state_dict_dmc_original_from_tf)
    return mobilenetv1_model
    # print(layer.name, type(layer))
mobilenetv1_torch_model = convert_mobilenetv1_tf_to_torch(save_weight=False)
type(mobilenetv1_torch_model)

NameError: name 'tf' is not defined